In [1]:
import spacy
# from spacy import displacy
from collections import Counter
# import en_core_web_sm
import pandas as pd

# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.tag import pos_tag

import re
# nlp = en_core_web_trf.load()
nlp = spacy.load("../custom-ner-with-spacy3/model-best36k_last") 

c:\ProgramData\Anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
# opening the file
file_obj = open("../zip/Breaking01.txt", "r",encoding="utf-8")
  
# reading the data from the file
file_data = file_obj.read()
  
# splitting the file data into lines
lines = file_data.splitlines()
# print(lines)
file_obj.close()

In [ ]:
id = []
# for i in range(len(lines)):
#     id.append(lines[i][:19])
#     lines[i] = lines[i][21:]
new_lines = []
j = 0
for i in range(len(lines)):
    j = j + 1
    if 'breaking' not in lines[i]:
        id.append(lines[i][:19])
        lines[i] = lines[i][21:]
        new_lines.append(lines[i])
lines = new_lines
df = pd.DataFrame()
df['text'] = lines
print(j)
len(lines)

54663


36019

In [ ]:
lines[92]

'BREAKING: @ElonMusk to form content moderation council with widely diverse viewpoints at Twitter and says:  “No major content decisions or account reinstatements will happen before that council convenes”'

In [ ]:
def deEmojify(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u20AC"
        u"\u23F3"
        u"\u2066"
        u"\u2069"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

In [ ]:
def get_1_mention(text):
    a = 0
    B = []
    words_list = text.split()
    for i in range(len(words_list)):
        if(words_list[i][0]=='@' and a != 1):
            B.append(words_list[i])
            a = 1
        if(words_list[i][0]!='@'):
            B.append(words_list[i])
            a = 0
    return ' '.join(B)

def get_1_hashtag(text):
    a = 0
    B = []
    words_list = text.split()
    for i in range(len(words_list)):
        if(words_list[i][0]=='#' and a != 1):
            B.append(words_list[i])
            a = 1
        if(words_list[i][0]!='#'):
            B.append(words_list[i])
            a = 0
    return ' '.join(B)

In [ ]:
text = df['text'].tolist()
text_clean = []
for i in range(len(text)):
    tw = re.sub(r'https\S+', '', text[i])
    tw = re.sub("\n", ' ', tw)
    # tw = re.sub("#", '', tw)
    # tw = re.sub("@[A-Za-z0-9_]+","", tw)
    tw = deEmojify(tw)
    tw = get_1_hashtag(tw)
    tw = get_1_mention(tw)
    tw = re.sub("@", '', tw)
    tw = tw.replace('[','')
    tw = tw.replace(']','')
    text_clean.append(tw)
# df['text'] = tw
# df

In [ ]:
for i in range(len(text_clean)):
    # lines[i] = re.sub("(?<=[a-zA-Z])'s", " 's", lines[i])
    # lines[i] = re.sub("(?<=[a-zA-Z])\.", " .", lines[i])
    text_clean[i] = re.sub("Breaking", "", text_clean[i])
    text_clean[i] = re.sub("BREAKING", "", text_clean[i])

In [ ]:
# list_of_string = []
# for i in range(len(lines)):
#     list_of_word = [(x.orth_) for x in [y for y in nlp(lines[i])]]
#     list_of_string.append(" ".join(list_of_word))
# lines = list_of_string

In [ ]:
len(lines)

36019

In [ ]:
ent_list = []
from tqdm import tqdm
for i in tqdm(text_clean):
    ex = nlp(i)
    enttity = []
    for ent in ex.ents:
        if (ent.label_ == 'PERSON' or ent.label_ == 'GPE' or ent.label_ == 'LOC' or ent.label_ == 'TIME' or 
            ent.label_ == 'DATE' or ent.label_ == 'ORG' or ent.label_ == 'NORP' or ent.label_ == 'FAC' 
            or ent.label_ == 'ACT' or ent.label_ == 'GPE_WHO'):
            enttity.append(ent.text.lower())
    ent_list.append(enttity)
    # print(enttity)
ent_list

100%|██████████| 36019/36019 [36:55<00:00, 16.26it/s]


[['republicans',
  'bear',
  'cleaning',
  'joe biden',
  'stay',
  'republican',
  'fight'],
 ['house', 'nancy pelosi', 'broken', 'assaulted'],
 ['nancy pelosi’s',
  'suspected',
  'targeted',
  'tell',
  'abc news',
  'entered',
  'sliding',
  'carrying',
  'looking',
  'house'],
 ['republican',
  'glenn youngkin',
  'responds',
  'paul pelosi',
  'saying',
  'gon',
  'na',
  'send',
  'nancy pelosi',
  'california',
  'rt',
  'stand',
  'pelosis'],
 ['paul pelosi',
  'undergoing',
  'is',
  'condemned',
  'marjorie taylor greene',
  'tucker carlson',
  'donald trump'],
 ['elon musk', 'fired', 'vijaya gadde', 'made', 'suspend', 'donald trump'],
 ["paul pelosi's", 'shouted', "'s", 'nancy'],
 ['azam khan', 'loses', 'uttar pradesh', 'sentenced', '3 year', 'follow'],
 ['nbc news',
  'confirms',
  'nancy pelosi’s',
  'paul pelosi',
  'undergoing',
  'following',
  'earlier today',
  'broke',
  'pelosi'],
 ['elon musk',
  'closed',
  'twitter',
  'parag agarawal',
  'ned segal',
  'vijaya 

In [ ]:
ex = text_clean[4]
spacy.displacy.render(nlp('Russia is country'), style="ent", jupyter=True) # display in Jupyter

In [ ]:
df['Entity'] = ent_list
df['Text clean'] = text_clean
df['id'] = id

In [ ]:
df

,text,Entity,Text clean,id
0,Why do republicans bear all the burden of clea...,"[republicans, bear, cleaning, joe biden, stay,...",Why do republicans bear all the burden of clea...,1586065712283807744
1,BREAKING: House Speaker Nancy Pelosi's house w...,"[house, nancy pelosi, broken, assaulted]",: House Speaker Nancy Pelosi's house was broke...,1586065712455421952
2,BREAKING: The break in at Nancy Pelosi’s house...,"[nancy pelosi’s, suspected, targeted, tell, ab...",: The break in at Nancy Pelosi’s house is susp...,1586065712510009346
3,BREAKING: Republican Gov. Glenn Youngkin disgu...,"[republican, glenn youngkin, responds, paul pe...",: Republican Gov. Glenn Youngkin disgustingly ...,1586065712728006656
4,BREAKING: Paul Pelosi is undergoing brain surg...,"[paul pelosi, undergoing, is, condemned, marjo...",: Paul Pelosi is undergoing brain surgery. The...,1586065714091204608
...,...,...,...,...
36014,Can we get more oil production please 🧡 BREAKI...,"[get, painted, 8:30am, today, sprayed, berkele...",Can we get more oil production please : LUXURY...,1586249871136358403
36015,Good https://t.co/zNYGqzExmW BREAKING: A feder...,"[friday, refused, bar, monitoring, arizona, ma...",Good : A federal judge Friday refused to bar a...,1586249886185689088
36016,Whoa Breaking: A new Senate report concludes t...,"[senate, concludes, causes, resulted, related,...",Whoa : A new Senate report concludes that SARS...,1586249894922137600
36017,Breaking News Alert: Digital News is a never-e...,[ending],News Alert: Digital News is a never-ending abyss,1586249950454632448


In [ ]:
name = "Breaking_36k"
file_name_path = './clean_ner/' + name + '.csv'
df.to_csv(file_name_path)

In [ ]:
df['text'][3]

'They hate anything beautiful and good 🚨 BREAKING: ROLEX STORE SPRAY-PAINTED ORANGE 🚨  ⌚️ At 8:30am today, two Just Stop Oil supporters sprayed orange paint from a fire extinguisher over the premises of @ROLEX in Knightsbridge, demanding that the government halts all new oil and gas consents and licences. #FreeLouis https://t.co/iBoNlQwTmc'

In [ ]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(str(df['text'][3]))]]

[('They', '', ''),
 ('hate', '', ''),
 ('anything', '', ''),
 ('beautiful', '', ''),
 ('and', '', ''),
 ('good', '', ''),
 ('🚨', '', ''),
 ('BREAKING', '', ''),
 (':', '', ''),
 ('ROLEX', '', ''),
 ('STORE', '', ''),
 ('SPRAY', '', ''),
 ('-', '', ''),
 ('PAINTED', '', ''),
 ('ORANGE', '', ''),
 ('🚨', '', ''),
 (' ', '', ''),
 ('⌚', '', ''),
 ('️', '', ''),
 ('At', '', ''),
 ('8:30am', '', ''),
 ('today', '', ''),
 (',', '', ''),
 ('two', '', ''),
 ('Just', '', ''),
 ('Stop', '', ''),
 ('Oil', '', ''),
 ('supporters', '', ''),
 ('sprayed', '', ''),
 ('orange', '', ''),
 ('paint', '', ''),
 ('from', '', ''),
 ('a', '', ''),
 ('fire', '', ''),
 ('extinguisher', '', ''),
 ('over', '', ''),
 ('the', '', ''),
 ('premises', '', ''),
 ('of', '', ''),
 ('@ROLEX', '', ''),
 ('in', '', ''),
 ('Knightsbridge', '', ''),
 (',', '', ''),
 ('demanding', '', ''),
 ('that', '', ''),
 ('the', '', ''),
 ('government', '', ''),
 ('halts', '', ''),
 ('all', '', ''),
 ('new', '', ''),
 ('oil', '', ''),
 ('a